In [1]:
from Pegasus.api import *

In [2]:
# not required. Just to view output from certain tools.
from pathlib import Path
import logging

logging.basicConfig(level=logging.DEBUG)

In [3]:
# creating input file number.txt 
with open("number.txt", "w") as f:
    f.write("10")

number = File("number.txt").add_metadata(creator="shubham")

In [8]:
rc = ReplicaCatalog()\
    .add_replica("local", number, Path(".").resolve() / "number")\
    .write()

tc = TransformationCatalog()

increment = Transformation(
                "increment",
                site="local",
                pfn="/home/scitech/shared-data/increment",
                is_stageable=True,
                arch=Arch.X86_64,
                os_type=OS.LINUX
)

tc.add_transformations(increment)
tc.write()

In [9]:
# create workflow

wf = Workflow("Pipeline")

result = File("result.txt")

job_increment = Job(increment)\
                .add_args("-a", "increment", "-T", "3", "-i", number, "-o", result)\
                .add_inputs(number)\
                .add_outputs(result)
wf.add_jobs(job_increment)

In [10]:
try:
    wf.plan(submit=True)\
        .wait()
except PegasusClientError as e:
    print(e)



################
# pegasus-plan #
################
2021.01.26 17:20:47.106 UTC:
2021.01.26 17:20:47.112 UTC:   -----------------------------------------------------------------------
2021.01.26 17:20:47.117 UTC:   File for submitting this DAG to HTCondor           : Pipeline-0.dag.condor.sub
2021.01.26 17:20:47.122 UTC:   Log of DAGMan debugging messages                 : Pipeline-0.dag.dagman.out
2021.01.26 17:20:47.131 UTC:   Log of HTCondor library output                     : Pipeline-0.dag.lib.out
2021.01.26 17:20:47.137 UTC:   Log of HTCondor library error messages             : Pipeline-0.dag.lib.err
2021.01.26 17:20:47.142 UTC:   Log of the life of condor_dagman itself          : Pipeline-0.dag.dagman.log
2021.01.26 17:20:47.147 UTC:
2021.01.26 17:20:47.153 UTC:   -no_submit given, not submitting DAG to HTCondor.  You can do this with:
2021.01.26 17:20:47.163 UTC:   -----------------------------------------------------------------------
2021.01.26 17:20:48.047 UTC:   Your data

[####--------------------------------]  11.1% ..Running (Completed: 1, Queued: 0, Running: 2, Failed: 0)
Cancelling Client.wait(). Your workflow is still running and can be monitored with pegasus-status
